# Validation SSL model

In this notebook our goal is to test how good our SSL pretrained weights are. 
- We will query images from different classes and compare embeddings. This will give us better insights for the intraclass/interclass variability.
    - Intraclass variance: variance within one class (The intraclass variance measures the differences between the individual embeddings within each class.)
    - Interclass variance: variance between different classes (The interclass variance measures the differences between the means of each class)
- Note: you need to run this notebook with a kernel in your venv to use vissl libs: https://janakiev.com/blog/jupyter-virtual-envs/#add-virtual-environment-to-jupyter-notebook

## Imports
- matplotlib for visualisation
- torch

In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

## Reading in pretrained weights

### Option 1: Imagenet pretrained
- Load the best imgnet pretrained weights, docs: https://pytorch.org/vision/stable/models.html
- This is currently ResNet50_Weights.IMAGENET1K_V2 with an accuracy of 80.858%
- weights are saved in /home/olivier/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


In [5]:
#imgnet weights
#model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
model = torchvision.models.resnet50(pretrained=True)
#torch.save(model.state_dict(),"resnet50_imgnet.pth")
#weights = torch.load("resnet50_imgnet.pth")
#print(weights.keys())
#print(model)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/olivier/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

### Option 2: SSL pretrained
Load weights from checkpoint according to vissl tutorial:
https://github.com/facebookresearch/vissl/blob/v0.1.6/tutorials/Using_a_pretrained_model_for_inference_V0_1_6.ipynb


In [3]:
#dictionary to summarize the paths to the the training config used and the path to the weigths
#train_config path is a relative path from the vissl folder
#weights path is an absolute path to where the final_checkpoint.torch is stored 
PATHS = {
    "rotnet":
    {
        "train_config": "validation/rotnet_full/train_config.yaml", #relative path from vissl/...
        "weights": "/home/olivier/Documents/master/mp/checkpoints/sku110k/rotnet_full/model_final_checkpoint_phase104.torch",
    },
    "jigsaw":
    {
        "train_config": "validation/jigsaw_full/train_config.yaml",
        "weights": "/home/olivier/Documents/master/mp/checkpoints/sku110k/jigsaw_full/model_final_checkpoint_phase104.torch"
    },
    "moco":
    {
        "train_config": "validation/moco_full/train_config.yaml",
        "weights": "/home/olivier/Documents/master/mp/checkpoints/sku110k/moco_full/model_final_checkpoint_phase199.torch"
    },
    "simclr":
    {
        "train_config": "",
        "weights": ""
    },
    "swav":
    {
        "train_config": "",
        "weights": ""
    }
    
}

#CHOOSE the model you want to validate here
train_config = PATHS["rotnet"]["train_config"] #change the key of the PATHS dict to the desired model name
weights_file = PATHS["rotnet"]["weights"]
print('Train config at (relative path from vissl/...):\n' + train_config)
print('SSL pretrained weights at:\n' + weights_file)

Train config at (relative path from vissl/...):
validation/rotnet_full/train_config.yaml
SSL pretrained weights at:
/home/olivier/Documents/master/mp/checkpoints/sku110k/rotnet_full/model_final_checkpoint_phase104.torch


In [4]:
from omegaconf import OmegaConf
from vissl.utils.hydra_config import AttrDict
from vissl.utils.hydra_config import compose_hydra_configuration, convert_to_attrdict

# 1. Checkpoint config is located at vissl/configs/config/validation/*/train_config.yaml.
# 2. weights are located at /home/olivier/Documents/master/mp/checkpoints/sku110k/*
# The * in the above paths stand for rotnet_full, jigsaw_full or moco_full
# All other options specified below override the train_config.yaml config.

cfg = [
  'config=' + train_config,
  'config.MODEL.WEIGHTS_INIT.PARAMS_FILE=' + weights_file, # Specify path for the model weights.
  'config.MODEL.FEATURE_EVAL_SETTINGS.EVAL_MODE_ON=True', # Turn on model evaluation mode.
  'config.MODEL.FEATURE_EVAL_SETTINGS.FREEZE_TRUNK_ONLY=True', # Freeze trunk. 
  'config.MODEL.FEATURE_EVAL_SETTINGS.EXTRACT_TRUNK_FEATURES_ONLY=True', # Extract the trunk features, as opposed to the HEAD.
  'config.MODEL.FEATURE_EVAL_SETTINGS.SHOULD_FLATTEN_FEATS=False', # Do not flatten features.
  'config.MODEL.FEATURE_EVAL_SETTINGS.LINEAR_EVAL_FEAT_POOL_OPS_MAP=[["res5avg", ["Identity", []]]]' # Extract only the res5avg features.
]

# Compose the hydra configuration.
cfg = compose_hydra_configuration(cfg)
# Convert to AttrDict. This method will also infer certain config options
# and validate the config is valid.
_, cfg = convert_to_attrdict(cfg)

** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



Now let's build the model with the exact training configs:

In [5]:
from vissl.models import build_model

model = build_model(cfg.MODEL, cfg.OPTIMIZER)

#### Loading the pretrained weights

In [6]:
from classy_vision.generic.util import load_checkpoint
from vissl.utils.checkpoint import init_model_from_consolidated_weights

# Load the checkpoint weights.
weights = load_checkpoint(checkpoint_path=cfg.MODEL.WEIGHTS_INIT.PARAMS_FILE)


# Initializei the model with the simclr model weights.
init_model_from_consolidated_weights(
    config=cfg,
    model=model,
    state_dict=weights,
    state_dict_key_name="classy_state_dict",
    skip_layers=[],  # Use this if you do not want to load all layers
)

print("Weights have loaded")

Weights have loaded


#### Extra info
- VISSL uses the ResNeXT50 class, which is their custom wrapper class
    - ResNeXT50 wrapper class is defined at https://github.com/facebookresearch/vissl/blob/04788de934b39278326331f7a4396e03e85f6e55/vissl/models/trunks/resnext.py
    - ResNet base class https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py for interface of the __init__ method.
    - the model of this wrapper class is a torchvision.models.ResNet() which we will reconstruct here based on the YAML config parameters.
- checkpoints from pretraining are stored on /home/olivier/Documents/master/mp/checkpoints/sku110k/
    - checkpoints have phase numbers: in VISSL, if the workflow involves training and testing both, the number of phases = train phases + test epochs. So if we alternate train and test, the phase number is: 0 (train), 1 (test), 2 (train), 3 (test)... and train_phase_idx is always: 0 (corresponds to phase0), 1 (correponds to phase 2)
    - The weights are stored 

In [7]:
print("Loading vissl checkpoint")
ssl_checkpoint = torch.load(Path(weights_file))
print("Checkpoint contains:")
dataframe_dict = dict()
dataframe_dict["phase_idx"] = ssl_checkpoint["phase_idx"]
dataframe_dict["iteration_num"] = ssl_checkpoint["iteration_num"]
dataframe_dict["train_phase_idx"] = ssl_checkpoint["train_phase_idx"]
dataframe_dict["iteration"] = ssl_checkpoint["iteration"]
dataframe_dict["type"] = ssl_checkpoint["type"]
df = pd.DataFrame(data=dataframe_dict.values(), index=dataframe_dict.keys(),columns=["Value"])
display(df)
if("loss", "classy_state_dict" in ssl_checkpoint.keys()):
    print("Checkpoint also contains elements loss and classy_state_dict")

#the weights of the trunk resnet network are stored in a nested dict:    
#print(ssl_checkpoint["classy_state_dict"]["base_model"]["model"]["trunk"].keys())

Loading vissl checkpoint


/home/olivier/Documents/master/mp/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Extracting features

In [7]:
from PIL import Image
import torchvision.transforms as transforms

def extract_features(path):
    image = Image.open(path)
    # Convert images to RGB. This is important
    # as the model was trained on RGB images.
    image = image.convert("RGB")

    # Image transformation pipeline.
    pipeline = transforms.Compose([
      transforms.CenterCrop(224),
      transforms.ToTensor(),
    ])
    x = pipeline(image)

    #unsqueeze adds a dim for batch size (with 1 element the entire input tensor of the image)
    features = model(x.unsqueeze(0))

    features_shape = features[0].shape
    #print(f"Features extracted have the shape: { features_shape }")
    return features[0]

In [8]:
savefile = open("fts.txt","w")
CornerShop = Path("/home/olivier/Documents/master/mp/CornerShop/CornerShop/crops")

#create an iterator over all jpg files in cornershop map and put elements in a list
img_paths = list(CornerShop.glob("*/*.jpg")) #**/*.jpg to look into all subdirs for jpgs and iterate over them
#extract the corresponding labels (folder names)
labels = [p.parent.stem for p,_ in zip(img_paths,range(20)) ] #stem attr, conatins foldername 
#path.stem=filename without extension
#path.name=filename with extension
fts_stack = torch.stack([extract_features(p).squeeze() for p,_ in zip(img_paths,range(20)) ])
print(fts_stack.shape)
print(labels[0:10])

torch.Size([20, 2048])
['CawstonDry', 'CawstonDry', 'CawstonDry', 'MinuteMaidAppelPerzik', 'CarrefourSmoothieAardbeiBlauweBessen', 'CarrefourSmoothieAardbeiBlauweBessen', 'CarrefourSmoothieAardbeiBlauweBessen', 'CarrefourSmoothieAardbeiBlauweBessen', 'GiniZeroFles1,5L', 'GiniZeroFles1,5L']


results of the feature extraction:
- fts_stack: contains n rows and 2048 columns (features), this is a stack of features from multiple query images
- labels: list with the corresponding labels of the feature stack

## Comparing features
Here we will investigate relations between the features from different images with:
- Inner product
- Cosine simularity
- Euclidian distance

### Inner product

In [9]:
#multiply the features of one tensor with all other tensors
inner_product = fts_stack.matmul(fts_stack.T)
print("Three examples from inner_product tensor:\n{}".format(inner_product[0:3]))

#data analytics
data = np.zeros((4))
#data
data[0]= inner_product.max()
data[1]= inner_product.min()
data[2]= torch.mean(inner_product)
data[3]= torch.std(inner_product)
#labels
data_labels=["max","min","avg","std_dev"]

df = pd.DataFrame(data=data,index=data_labels, columns=["inner_product"])
display(df)

Three examples from inner_product tensor:
tensor([[ 87.3110,  83.9168,  87.3473, 109.3133, 135.7349,  77.9418,  95.8140,
         120.6644,  75.7141, 139.5700,  75.3643, 109.6743, 179.2607,  81.3136,
         126.0925,  98.3647, 115.1595,  96.6702, 104.6836,  94.0268],
        [ 83.9168,  88.7915,  87.2473, 110.2363, 137.6209,  79.6288,  97.1384,
         122.2782,  77.9874, 142.5118,  77.8924, 110.9518, 181.8811,  82.6621,
         127.6149, 100.1078, 115.6713,  97.8970, 105.6707,  95.2109],
        [ 87.3473,  87.2473,  93.0548, 112.7572, 141.2955,  81.3216,  99.6091,
         125.5396,  79.4377, 145.5551,  79.1773, 114.1908, 187.3758,  84.7680,
         131.0218, 102.5855, 119.3287, 100.4800, 108.6117,  97.5758]])


,inner_product
max,417.249237
min,71.108810
avg,132.755661
std_dev,47.856178


### Cosine simularity
Here we normalize the features and then calculate the inner product with all other tensors.

In [10]:
#NORMALIZE features in feature stack:
fts_stack_norm = fts_stack / fts_stack.norm(dim=1,keepdim=True) 
#fts.norm(dim=1,keepdim=True)
# dim=1: calculate norm over the second dimension (features/columns)
# keepdim=True: keep batch/stack dimension of features

#.norm is deprecated, newer version https://pytorch.org/docs/stable/generated/torch.linalg.matrix_norm.html#torch.linalg.matrix_norm 
#fts_stack_norm = fts_stack / torch.linalg.matrix_norm(fts_stack, dim=1, keepdim=True) #newer version?

In [11]:
#calculate cosine simularity (cosim)
#fts_stack is a matrix with n rows and 2048 columns (features)
#matrix product of fts_stack * fts_stack^T = cosin_sim with all other images from the stack
cosim = fts_stack_norm.matmul(fts_stack_norm.T)
print("Three examples from cosim tensor:\n{}".format(cosim[0:3]))

#data analytics
data = np.zeros((4))
#data
data[0]= cosim.max()
data[1]= cosim.min()
data[2]= torch.mean(cosim)
data[3]= torch.std(cosim)
#labels
data_labels=["max","min","avg","std_dev"]

df = pd.DataFrame(data=data,index=data_labels, columns=["cosim"])
display(df)

Three examples from cosim tensor:
tensor([[1.0000, 0.9531, 0.9690, 0.9594, 0.9534, 0.9452, 0.9551, 0.9512, 0.9435,
         0.9468, 0.9376, 0.9559, 0.9392, 0.9491, 0.9513, 0.9513, 0.9578, 0.9568,
         0.9569, 0.9572],
        [0.9531, 1.0000, 0.9598, 0.9594, 0.9585, 0.9576, 0.9602, 0.9559, 0.9637,
         0.9587, 0.9609, 0.9589, 0.9449, 0.9567, 0.9547, 0.9601, 0.9540, 0.9608,
         0.9578, 0.9611],
        [0.9690, 0.9598, 1.0000, 0.9586, 0.9613, 0.9553, 0.9618, 0.9586, 0.9589,
         0.9565, 0.9542, 0.9640, 0.9509, 0.9584, 0.9575, 0.9611, 0.9613, 0.9633,
         0.9617, 0.9621]])


,cosim
max,1.000001
min,0.931526
avg,0.961039
std_dev,0.012083


### Euclidean distance

In [12]:
eucl_dist = [] 
for tensor in fts_stack:
    d = [] #store all distances from this tensor to all the other tensors
    for other_tensor in fts_stack:
        d_to = (tensor - other_tensor).pow(2).sum().sqrt() #d(tensor, other_tensor)=euclid distance
        d.append(d_to)
    d = torch.tensor(d)
    #print("distance tensor has shape {}".format(d.shape))
    #add tensor to euclidian distances 
    eucl_dist.append(d)
eucl_dist = torch.stack(eucl_dist)
#print("eucl_dist has shape {}".format(eucl_dist.shape))
print("Three examples from eucl_dist tensor:\n{}".format(eucl_dist[0:3]))

#data analytics
data = np.zeros((4))
#data
data[0]= eucl_dist.max()
data[1]= eucl_dist.min()
data[2]= torch.mean(eucl_dist)
data[3]= torch.std(eucl_dist)
#labels
data_labels=["max","min","avg","std_dev"]

df = pd.DataFrame(data=data,index=data_labels, columns=["eucl_dist"])
display(df)

Three examples from eucl_dist tensor:
tensor([[ 0.0000,  2.8756,  2.3814,  4.1689,  6.9285,  3.0503,  3.3094,  5.5032,
          3.1040,  7.5526,  3.2529,  4.3291, 12.0847,  2.9595,  6.0288,  3.6089,
          4.7509,  3.2993,  3.8750,  3.1279],
        [ 2.8756,  0.0000,  2.7114,  4.1249,  6.7611,  2.7222,  3.1279,  5.3422,
          2.5630,  7.2552,  2.6469,  4.2031, 11.9281,  2.7462,  5.8977,  3.3195,
          4.7987,  3.1484,  3.8108,  2.9826],
        [ 2.3814,  2.7114,  0.0000,  4.0294,  6.5289,  2.8789,  3.0175,  5.1263,
          2.8163,  7.1285,  2.9495,  3.9309, 11.6427,  2.7556,  5.6773,  3.2135,
          4.4695,  3.0016,  3.5922,  2.9034]])


,eucl_dist
max,12.801560
min,0.000000
avg,4.611862
std_dev,2.495889


### Euclidian distance (normalized features)
Using normalized features

In [15]:
eucl_dist_norm = [] 
for tensor in fts_stack_norm:
    d = [] #store all distances from this tensor to all the other tensors
    for other_tensor in fts_stack_norm:
        d_to = (tensor - other_tensor).pow(2).sum().sqrt() #d(tensor, other_tensor)=euclid distance
        d.append(d_to)
    d = torch.tensor(d)
    #print("distance tensor has shape {}".format(d.shape))
    #add tensor to euclidian distances 
    eucl_dist_norm.append(d)
eucl_dist_norm = torch.stack(eucl_dist_norm)
#print("eucl_dist has shape {}".format(eucl_dist.shape))
print("Three examples from eucl_dist tensor:\n{}".format(eucl_dist_norm[0:3]))

#data analytics
data = np.zeros((4))
#data
data[0]= eucl_dist_norm.max()
data[1]= eucl_dist_norm.min()
data[2]= torch.mean(eucl_dist_norm)
data[3]= torch.std(eucl_dist_norm)
#labels
data_labels=["max","min","avg","std_dev"]

df = pd.DataFrame(data=data,index=data_labels, columns=["eucl_dist_norm"])
display(df)

Three examples from eucl_dist tensor:
tensor([[0.0000, 0.3063, 0.2488, 0.2850, 0.3054, 0.3310, 0.2997, 0.3124, 0.3361,
         0.3261, 0.3533, 0.2971, 0.3487, 0.3192, 0.3121, 0.3119, 0.2906, 0.2939,
         0.2936, 0.2927],
        [0.3063, 0.0000, 0.2834, 0.2850, 0.2880, 0.2912, 0.2822, 0.2971, 0.2694,
         0.2874, 0.2795, 0.2867, 0.3318, 0.2942, 0.3009, 0.2825, 0.3034, 0.2799,
         0.2904, 0.2789],
        [0.2488, 0.2834, 0.0000, 0.2878, 0.2782, 0.2990, 0.2765, 0.2877, 0.2867,
         0.2951, 0.3028, 0.2682, 0.3133, 0.2886, 0.2916, 0.2791, 0.2781, 0.2708,
         0.2768, 0.2752]])


,eucl_dist_norm
max,0.370063
min,0.000000
avg,0.270670
std_dev,0.068351
